In [45]:
import torch
import torch.nn as nn

torch.set_default_device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50_257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,
}

In [3]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx: torch.Tensor):
        batch_size, seq_len = in_idx.shape

        # ["hello", "world"] -> [4.394, 1.685]
        tok_emb = self.tok_emb(in_idx)

        # [0, 1] -> [2.793, 1.164]
        pos_emb = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        x = tok_emb + pos_emb

        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.out_head(x)

        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x



In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x: torch.Tensor):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]], device='cuda:0')


In [6]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)
tokenizer.decode_single_token_bytes(logits[0][-1].argmax())

Output shape:  torch.Size([2, 4, 50257])
tensor([[[ 0.3227, -0.1641,  0.9602,  ...,  0.4737, -2.4280,  0.5969],
         [ 0.9188,  1.1286,  0.0131,  ..., -0.6804, -0.2360,  0.5531],
         [-0.7444,  0.1130, -0.6241,  ..., -0.5063, -0.7382, -0.1368],
         [ 1.2785, -0.0472, -0.1038,  ..., -0.0546, -0.6380, -0.0660]],

        [[ 0.1904, -0.2022,  1.0400,  ...,  0.8918, -1.6030,  0.9570],
         [-0.2194,  1.0844, -0.7797,  ..., -0.2095, -1.9934,  1.1164],
         [-0.2034,  1.0729, -0.5321,  ..., -1.7061,  1.3856, -1.2117],
         [ 0.9472, -0.2335, -1.2444,  ..., -1.4038, -0.8587, -0.5734]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)


b' rant'

In [44]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
# print(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)
